Install smartcrop into google colab

MUST RESTART RUNTIME AFTER RUNNING THIS CELL

In [0]:
!pip install -e git+git://github.com/hhatto/smartcrop.py.git@master#egg=smartcrop

Obtaining smartcrop from git+git://github.com/hhatto/smartcrop.py.git@master#egg=smartcrop
  Cloning git://github.com/hhatto/smartcrop.py.git (to revision master) to ./src/smartcrop
  Running command git clone -q git://github.com/hhatto/smartcrop.py.git /content/src/smartcrop
  Running setup.py develop for smartcrop


Mount google drive

In [0]:
from google.colab import drive
drive.mount('drive', force_remount=True)

Mounted at drive


Import libraries

In [0]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing import image as Im
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
import glob
import os,imageio
import json
import sys
from PIL import Image


import smartcrop
import cv2

Set image root directory

In [0]:
root = "drive/My Drive/croptest/"
sc = smartcrop.SmartCrop()

Mapping and weight adjustment functions

In [0]:
def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

def within2(value):
  if value <= 2 and value >= -2:
    return 1
  else: return 0


def adjust_weights(ss_zscore, weights):
    #outputs array of elements with 1 or 0 if the entry is within 2 std
    check_within = list(map(within2, ss_zscore))
    #determine number of entries that are within 2 std
    count = sum(check_within)
    # all entries are within the std range
    if count == 0:
      #how to deal with weight when all are within range we expect/want
      return weights
    else:
      values = np.zeros(numSensors)
      for i in range(0, numSensors):
        #if the value is not within 2 std
        if not check_within[i]:
          #set temp arr to value deducted at index i
          new_weight = (weights[i] * (-2/ss_zscore[i]))
          values[i] = weights[i] - new_weight
          weights[i] = new_weight
      #change weight of sensor value that is more than 2 std from mean
      numToAdd = sum(values) / count
      for i in range(0, numSensors):
        #if the entry is within 2 std
        if check_within[i]:
          weights[i] = weights[i] + numToAdd
      return weights


Define constants - 

set hardware constants

In [0]:
lensfocallength = 1.0
fstop = 2.8

aperture = lensfocallength/fstop
ISO = 1
time = 1
sensor_max = 1000
sensor_min = 100


camera_scale = (np.sqrt(2)**aperture) / (ISO * time)

numSensors = 4

#initial weights
weights = [0.25, 0.25, 0.25, 0.25]
#random values for now
ss_arr = [800, 701, 580, 511]



mean: 711.5845097109011 std_dev: 99.78389414728649

Top-level function

In [0]:
def process(image):
  #converts to RBG
  img = Image.open(image).convert("RGB")

  #determine best crop information
  best_crop = sc.crop (img, 72, 72)
  best_crop_x = best_crop["top_crop"]['x']
  best_crop_y = best_crop["top_crop"]['y']
  width = best_crop["top_crop"]['width']
  height = best_crop["top_crop"]['height']

  #crop actual image
  im_crop = img.crop((best_crop_x,best_crop_y, best_crop_x+width, best_crop_y+height))

  #show cropped images
  #plt.imshow(im_crop)
  #plt.figure()

  #convert to numpy array and preserve 3 channels
  image_3ch = np.asarray(im_crop)

  #convert to Luv Image format
  im_cv = cv2.cvtColor(image_3ch, cv2.COLOR_RGB2LUV)
  #extract light from Luv
  value_arr = im_cv[:,:,0]
  #Scale light values based on sensor values
  reshape_arr = (value_arr.flatten() * camera_scale)
  arr_size = len(reshape_arr)
  #CVzScore = (reshape_arr-np.mean(reshape_arr))/np.std(reshape_arr)

  #map all values to sensor
  arr_sensorCV = np.zeros(arr_size)
  for i in range(0, arr_size):
    arr_sensorCV[i] = translate(reshape_arr[i], 0, 255, sensor_min, sensor_max)
  #print(arr_sensorCV)
  #print("mapped values: \n" + str(arr_sensorCV))


  #determine the mean and std of Camera array scaled
  sensorCV_mean = np.mean(arr_sensorCV)
  sensorCV_stddev = np.std(arr_sensorCV)

  print("mean: " + str(sensorCV_mean) + " std_dev: " + str(sensorCV_stddev))
  #print("sensor values: " + str(ss_arr))
  #determine zscore of sunsensor readings based on mean and std of camera
  ss_zscore = (ss_arr - sensorCV_mean)/sensorCV_stddev

  new_weights = adjust_weights(ss_zscore, weights)
  print("New Weights: " + str(new_weights))

Code to convert video to images

In [0]:
vidcap = cv2.VideoCapture(root+"demo/Dandelion.mp4")
success,image = vidcap.read()
count = 0;
while success:
  cv2.imwrite(root+"demo/frame%d.jpg" % count,image)
  success,image = vidcap.read()
  #print('read a new frame: ', success)
  count+=1

Run to process all images

1. Change the directory of images
2. Delete the test values, used only for demonstration purposes

In [0]:
images = glob.glob(root+"smalltest/*.jpg")
#image = Image.open(root+"/smalltest/daisy0003.jpg", "r")

#Reset weights
weights = [0.25, 0.25, 0.25, 0.25]

#These values are just for the demo, values should come from sensor outputs
test= [[800, 701, 580, 400],[798, 710, 600, 300], [803, 708, 597, 98],[803, 708, 597, 490], [803, 708, 597, 240], [803, 708, 597, 410], [803, 708, 597, 498],[803, 708, 597, 400],[803, 708, 597, 380],[803, 708, 597, 498],[803, 708, 597, 350]]



for image in images:
  for val in test:
    ss_arr = val
    process(image)



mean: 711.5845097109011 std_dev: 99.78389414728649
New Weights: [0.27995889235108345, 0.27995889235108345, 0.27995889235108345, 0.16012332294674958]
mean: 711.5845097109011 std_dev: 99.78389414728649
New Weights: [0.307453304112193, 0.307453304112193, 0.307453304112193, 0.07764008766342088]
mean: 711.5845097109011 std_dev: 99.78389414728649
New Weights: [0.3249158780180168, 0.3249158780180168, 0.3249158780180168, 0.02525236594594948]
mean: 711.5845097109011 std_dev: 99.78389414728649
New Weights: [0.32575223958929045, 0.32575223958929045, 0.32575223958929045, 0.022743281232128487]
mean: 711.5845097109011 std_dev: 99.78389414728649
New Weights: [0.33012512339386785, 0.33012512339386785, 0.33012512339386785, 0.009624629818396329]
mean: 711.5845097109011 std_dev: 99.78389414728649
New Weights: [0.33121036169060025, 0.33121036169060025, 0.33121036169060025, 0.006368914928199062]
mean: 711.5845097109011 std_dev: 99.78389414728649
New Weights: [0.33134968406997706, 0.33134968406997706, 0.331